In [ ]:
import numpy as np
import holoviews as hv
hv.extension('matplotlib')

from solversuperres import *

In [ ]:
plots = dict()

In [ ]:
k = 100
s_true = Signal_true(d=2, k=k, m=k * 40, eps_dist_spikes=0.015)
s_true.init_spikes(manual_a=np.random.rand(s_true.k) * 4 + 1)

s_esti = s_true.create_estimation(coef_k=5)

In [ ]:
solver = Solver(s_true, s_esti)

In [ ]:
s_true.init_mesures(c=0.02)

In [ ]:
grid = Grid(201, 2)

In [ ]:
zy = Aty(solver.s_true.y, solver.s_true.w, grid.coords)

In [ ]:
plot_s_true_energy = (plot.energy2D(zy).opts(clabel='Energy') * plot.true_points(solver.s_true).opts(colorbar=False, fontscale=2)).opts(legend_position='top_right')
plots['true_points_energy'] = plot_s_true_energy
hv.output(plot_s_true_energy)
# plot.energy2D(zy) * plot.true_points(s_true)

## COMP

In [ ]:
%%time
solver.initialize(opt='COMP', minimizer='cg', tau={'min': -7, 'max': 0}, nit=500, nb_tests=20)

In [ ]:
COMP_esti_true_points = (plot.esti_points(solver.s_esti) * plot.true_points(solver.s_true).opts(fontscale=2)).opts(legend_position='top_right')
plots['COMP_esti_true_points'] = COMP_esti_true_points
hv.output(COMP_esti_true_points)

In [ ]:
COMP_error_init = plot.error(solver, alg='COMP', logy=True).opts(fontscale=2, aspect=0.5)
plots['COMP_error_init'] = COMP_error_init
hv.output(COMP_error_init)

In [ ]:
r = solver.s_true.y - Ax(solver.s_esti.a_init, solver.s_true.w, solver.s_esti.t_init)
zr = Aty(r, solver.s_true.w, grid.coords)

In [ ]:
COMP_before_energy = plot.energy2D(zy).opts(clabel='Energy', fontscale=2)
COMP_after_energy  = plot.energy2D(zr).opts(clabel='Energy', fontscale=2)
plots['COMP_before_energy'] = COMP_before_energy
plots['COMP_after_energy'] = COMP_after_energy
hv.output((COMP_before_energy + COMP_after_energy).opts(shared_axes=False))

In [ ]:
COMP_matrix_condition = plot.condition(solver, alg='COMP').opts(fontscale=2, aspect=1)
plots['COMP_matrix_condition'] = COMP_matrix_condition
hv.output(COMP_matrix_condition)

## SCOMP

In [ ]:
%%time
solver.initialize(opt='SCOMP', minimizer='cg', tau={'min': -7, 'max': 0}, min_iter=350, nb_tests=1)

In [ ]:
color_range = np.arange(255, -1, -1)
red_cmap = list(map(lambda c: f'#ff{c:02x}{c:02x}', color_range))
green_cmap = list(map(lambda c: f'#{c:02x}ff{c:02x}', color_range))
blue_cmap = list(map(lambda c: f'#{c:02x}{c:02x}ff', color_range))

diverging_cmap = green_cmap[::-1][:192] + blue_cmap[64:]
SCOMP_init_true_points = (plot.esti_points(solver.s_esti).opts(cmap=diverging_cmap) * plot.true_points(solver.s_true)).opts(fontscale=2, legend_position='top_right')
plots['SCOMP_init_true_points'] = SCOMP_init_true_points
hv.output(SCOMP_init_true_points)

In [ ]:
r = solver.s_true.y - Ax(solver.s_esti.a_init, solver.s_true.w, solver.s_esti.t_init)
zr = Aty(r, solver.s_true.w, grid.coords)

In [ ]:
SCOMP_before_init_energy = plot.energy2D(zy).opts(clabel='Energy', fontscale=2)
SCOMP_after_init_energy  = plot.energy2D(zr).opts(clabel='Energy', fontscale=2)
plots['SCOMP_before_init_energy'] = SCOMP_before_init_energy
plots['SCOMP_after_init_energy'] = SCOMP_after_init_energy
hv.output((SCOMP_before_init_energy + SCOMP_after_init_energy).opts(shared_axes=False))

In [ ]:
SCOMP_error = plot.error(solver, alg='SCOMP', logy=True).opts(fontscale=2, aspect=0.5)
plots['SCOMP_error_init'] = SCOMP_error
hv.output(SCOMP_error)

In [ ]:
SCOMP_matrix_condition = plot.condition(solver, alg='SCOMP').opts(fontscale=2, aspect=1)
plots['SCOMP_matrix_condition'] = SCOMP_matrix_condition
hv.output(SCOMP_matrix_condition)

In [ ]:
%%time
solver.optimize(eps_proj=0.015, project=True, tau={'min': -8, 'max': 1}, nit=3_000)

In [ ]:
SCOMP_esti_true_points = (plot.esti_points(solver.s_esti) * plot.true_points(solver.s_true)).opts(fontscale=2,legend_position='top_right')
plots['SCOMP_esti_true_points'] = SCOMP_esti_true_points
hv.output(SCOMP_esti_true_points)

In [ ]:
SCOMP_error_esti = plot.error(solver, alg='PGD', logy=True).opts(aspect=0.5, fontscale=2)
plots['SCOMP_error_esti'] = SCOMP_error_esti
hv.output(SCOMP_error_esti)

In [ ]:
r = solver.s_true.y - Ax(solver.s_esti.a_opt, solver.s_true.w, solver.s_esti.t_opt)
zr = Aty(r, solver.s_true.w, grid.coords)

In [ ]:
SCOMP_before_esti_energy = plot.energy2D(zy).opts(clabel='Energy', fontscale=2)
SCOMP_after_esti_energy  = plot.energy2D(zr).opts(clabel='Energy', fontscale=2)
plots['SCOMP_before_esti_energy'] = SCOMP_before_esti_energy
plots['SCOMP_after_esti_energy'] = SCOMP_after_esti_energy
hv.output((SCOMP_before_esti_energy + SCOMP_after_esti_energy).opts(shared_axes=False))

## Saving plots

In [ ]:
def save_svg(directory, dict_plots):
    for name, plot in dict_plots.items():
        hv.save(plot, f'{directory}/{name}', fmt='svg')

In [ ]:
save_svg('imgs', plots)